In [ ]:
#pip install ffmpeg-python

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math, json, os
import torch
import itertools
from sentence_transformers import SentenceTransformer
import ffmpeg


/Users/purnimakamath/opt/anaconda3/envs/video-summaries/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
def checknum(a):
    try:
        int(a)
    except:
        return False
    return True

In [64]:
with open("videos/NUSFutureWork-Deepgram.json") as f:
    deepgram_json = json.load(f)
f.close()

transcript = deepgram_json['results']['channels'][0]['alternatives'][0]['transcript']
paragraphs = deepgram_json['results']['channels'][0]['alternatives'][0]['paragraphs']
sentences_obj = [_ for _ in list(itertools.chain(*[para['sentences'] for para in paragraphs['paragraphs']]))] # needed for start and end times
sentences = [_['text'] for _ in list(itertools.chain(*[para['sentences'] for para in paragraphs['paragraphs']]))]
sentence_lengths = [int((_['end']-_['start'])*1) for _ in list(itertools.chain(*[para['sentences'] for para in paragraphs['paragraphs']]))]
#summary
# script = ''
# for sentence in sentences:
#     script += sentence
# print(script)

len(sentences), len(sentence_lengths)

(79, 79)

In [65]:
for sentence in sentences:
    print(sentence)

Ladies and gentlemen, please put your hands together to welcome our guest of honor, minister for education, mister Chan Chun Sing.
A very good morning to guest of honor, minister for education, mister Chanchin Singh.
Keynote speaker, professor James Landay.
NUS president, professor Tan Eng Chai.
Deputy president of academic affairs and provost, professor Aaron Tian.
Distinguished guests, alumni, and partners here at NUSS and tuning in online.
Welcome to Future Work 2.0, the second edition of the NUS Lifelong Learning Festival.
My name is Joey, and I'm a proud alumna, a staff member of NUS, and your emcee for this morning.
The theme for this year's conference is AI in action, and we are grateful that so many industry leaders and experts have generously offered their time to get today to share their invaluable experiences in this area.
We are also grateful that you have taken the time to attend the conference, and we hope that you will leave with insights, connections, and inspiration fo

# Summarize this transcript "<contents>". In the summary, use very short sentences. Each sentence must be a distinct topic in the transcript. Do not use abbreviations and titles.

In [66]:
chat_gpt_summary =  "The event welcomed attendees to the second National University of Singapore Lifelong Learning Festival, themed Future Work 2 point 0: AI in Action. The opening remarks emphasized the importance of embracing artificial intelligence's transformative potential across industries. The University’s President highlighted the growing impact of artificial intelligence, likening it to earlier technological revolutions like personal computers and the internet. The University has been proactive in integrating artificial intelligence across its programs. It launched the National University of Singapore AI Institute to foster research and collaborations with industry and government. The institution also developed policies and training initiatives to guide staff and students on using artificial intelligence in education, including workshops on generative AI for course design and assessment. Moreover, the University’s administrative departments use artificial intelligence to enhance productivity. Staff training in data literacy and artificial intelligence has reached thousands, preparing employees for AI-driven tasks. The University also supports lifelong learning through a wide selection of courses, certificates, and modular learning options in artificial intelligence and other fields. Recognizing the cost of education as a barrier, the University announced a 40% tuition fee rebate for Singaporeans and permanent residents on over 80 master’s programs. The speaker concluded with a call to use artificial intelligence to empower, rather than replace, human workers. The event closed with thanks to the School of Continuing and Lifelong Learning and all participants."
# chat_gpt_summary = "The event opened with a welcome for the Minister for Education, Chan Chun Sing. Joey, the emcee, introduced the NUS Lifelong Learning Festival, Future Work 2 point 0. This year's theme is AI in Action, emphasizing AI's practical applications. NUS President Tan Eng Chai highlighted AI's transformative impact on work. He noted the AI Institute's establishment for research and collaboration. NUS is integrating AI into education and administration. The Center for Teaching, Learning, and Technology offers AI workshops. NUS IT has developed a generative AI platform for staff queries. Over 5,000 administrative staff completed data and AI courses. NUS offers numerous AI-related courses for lifelong learning. A 40% tuition rebate on 80 master’s programs was announced for Singapore citizens and residents. Flexible course formats are available to support continuous education. Tan encouraged learning to leverage AI for productivity. He thanked organizers and participants for supporting the festival."
chatgpt_summary_sentences = []

for sentence in chat_gpt_summary.split("."):
    if len(sentence) > 0:
        chatgpt_summary_sentences.append(sentence.strip()+".")
        

In [67]:
chatgpt_summary_sentences

['The event welcomed attendees to the second National University of Singapore Lifelong Learning Festival, themed Future Work 2 point 0: AI in Action.',
 "The opening remarks emphasized the importance of embracing artificial intelligence's transformative potential across industries.",
 'The University’s President highlighted the growing impact of artificial intelligence, likening it to earlier technological revolutions like personal computers and the internet.',
 'The University has been proactive in integrating artificial intelligence across its programs.',
 'It launched the National University of Singapore AI Institute to foster research and collaborations with industry and government.',
 'The institution also developed policies and training initiatives to guide staff and students on using artificial intelligence in education, including workshops on generative AI for course design and assessment.',
 'Moreover, the University’s administrative departments use artificial intelligence to 

In [54]:
# Sentence Scores

In [9]:
# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

In [68]:
chatgpt_embeddings = model.encode(chatgpt_summary_sentences)
sentences_embeddings = model.encode(sentences)

In [69]:
similarities = model.similarity(chatgpt_embeddings, sentences_embeddings) # Not used
for ind1, similarity in enumerate(similarities):
    for ind2, s in enumerate(similarity):
        # similarities[ind1][ind2] = s * sentence_lengths[ind2]# Dont do this. Sentence len is skewing up things.
        similarities[ind1][ind2] = s 

cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
cos_similarities = []
for chatgpt_embedding in chatgpt_embeddings:
    chatgpt_sim = []
    for ind, sentences_embedding in enumerate(sentences_embeddings):
        # chatgpt_sim.append(cos(torch.from_numpy(chatgpt_embedding), torch.from_numpy(sentences_embedding)).numpy()*sentence_lengths[ind]) # Dont do this. Sentence len is skewing up things.
        chatgpt_sim.append(cos(torch.from_numpy(chatgpt_embedding), torch.from_numpy(sentences_embedding)).numpy()*sentence_lengths[ind])
    cos_similarities.append(chatgpt_sim)

In [70]:
len(similarities), len(cos_similarities)

(12, 12)

In [71]:
for i in range(len(chatgpt_summary_sentences)):
    print(chatgpt_summary_sentences[i], '==>', \
          sentences[np.argmax(similarities[i])], \
          similarities[i][np.argmax(similarities[i])], \
          sentence_lengths[np.argmax(similarities[i])])
    print('--------')

The event welcomed attendees to the second National University of Singapore Lifelong Learning Festival, themed Future Work 2 point 0: AI in Action. ==> The theme for this year's learning festival is Future Work 2.0, AI in Action. tensor(0.7325) 7
--------
The opening remarks emphasized the importance of embracing artificial intelligence's transformative potential across industries. ==> It may sound like a paradox, but AI is for real and in action, transforming work and industries. tensor(0.6801) 8
--------
The University’s President highlighted the growing impact of artificial intelligence, likening it to earlier technological revolutions like personal computers and the internet. ==> And this brings together AI researchers from across the university to accelerate frontier AI research as well as to boost real world impact. tensor(0.6425) 12
--------
The University has been proactive in integrating artificial intelligence across its programs. ==> NUS has also been proactively guiding and

In [72]:
len(cos_similarities), len(chatgpt_summary_sentences), len(sentence_lengths), len(sentences)

(12, 12, 79, 79)

In [86]:
capacity = 30 #40 seconds | capacity

#knapsack problem

In [87]:
items = [] # sentences
weights = [] # sentence lengths
values = [] # similarities

item_starts = [] #start and stop times in video
item_ends = []

for i in range(len(chatgpt_summary_sentences)):
    items.append(sentences[np.argmax(similarities[i])])
    weights.append(sentence_lengths[np.argmax(similarities[i])])
    values.append(similarities[i][np.argmax(similarities[i])])

    item_starts.append(sentences_obj[np.argmax(similarities[i])]['start'])
    item_ends.append(sentences_obj[np.argmax(similarities[i])]['end'])


In [88]:
n = len(items)
dp = [[0 for i in range(capacity+1)] for w in range(n+1)] #15 X 401

In [89]:
for i in range(n+1):
    for w in range(capacity+1):
        if i==0 or w==0:
            dp[i][w] = 0
        elif weights[i-1] <= w:
            dp[i][w] = max(values[i-1]+dp[i-1][w-weights[i-1]], dp[i-1][w])
        else:
            dp[i][w] = dp[i-1][w]
    w = capacity
    chosen_items = []

    for i in range(n, 0, -1):
        if dp[i][w] != dp[i-1][w]:
            chosen_items.append(items[i-1])
            w -= weights[i-1]
    total_value = dp[n][capacity]

In [90]:
chosen_items[::-1], total_value

(["The theme for this year's learning festival is Future Work 2.0, AI in Action.",
  'It may sound like a paradox, but AI is for real and in action, transforming work and industries.',
  'Without much ado, I would like to thank the school of continuing and lifelong learning for putting this event together.'],
 tensor(2.0718))

In [83]:
probe = ffmpeg.probe('videos/NUS FutureWork.mp4')
video_info = next(s for s in probe['streams'] if s['codec_type'] == 'video')
fps = int(video_info['r_frame_rate'].split('/')[0])
fps, video_info['r_frame_rate']

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


(25, '25/1')

In [84]:
input_file = ffmpeg.input('videos/NUS FutureWork.mp4')


tot = 0
for i in range(len(chosen_items)):
    snip = sentences_obj[sentences.index(chosen_items[::-1][i])]
    tot += (snip['end']-snip['start'])
    print(snip['start']*fps, snip['end']*fps)

    out_temp = 'videos/output_'+str(i)+'.mp4'
    if os.path.exists(out_temp):
        os.remove(out_temp)
        
    pts='PTS-STARTPTS'
    vid = input_file.trim(start=snip['start'], end=snip['end']).setpts(pts)
    aud = (input_file
          .filter_('atrim', start=snip['start'], end=snip['end'])
          .filter_('asetpts', pts))
    
    vid_aud = ffmpeg.concat(vid, aud, v=1, a=1)
    output_file = ffmpeg.output(vid_aud, out_temp, format='mp4')
    ffmpeg.run(output_file)
print(tot)

4603.625 4796.125


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ffmpeg version 4.1.4 Copyright (c) 2000-2019 the FFmpeg developers
  built with Apple LLVM version 10.0.1 (clang-1001.0.46.4)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.1.4_1 --enable-shared --enable-pthreads --enable-version3 --enable-avresample --cc=clang --host-cflags='-I/Library/Java/JavaVirtualMachines/adoptopenjdk-12.0.1.jdk/Contents/Home/include -I/Library/Java/JavaVirtualMachines/adoptopenjdk-12.0.1.jdk/Contents/Home/include/darwin' --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libmp3lame --enable-libopus --enable-librubberband --enable-libsnappy --enable-libtesseract --enable-libtheora --ena

5089.49975 5298.375


ffmpeg version 4.1.4 Copyright (c) 2000-2019 the FFmpeg developers
  built with Apple LLVM version 10.0.1 (clang-1001.0.46.4)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.1.4_1 --enable-shared --enable-pthreads --enable-version3 --enable-avresample --cc=clang --host-cflags='-I/Library/Java/JavaVirtualMachines/adoptopenjdk-12.0.1.jdk/Contents/Home/include -I/Library/Java/JavaVirtualMachines/adoptopenjdk-12.0.1.jdk/Contents/Home/include/darwin' --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libmp3lame --enable-libopus --enable-librubberband --enable-libsnappy --enable-libtesseract --enable-libtheora --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librtmp --enable-libspeex --enable-videotoolbox --disable-libjack --d

13425.499 13828.624


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ffmpeg version 4.1.4 Copyright (c) 2000-2019 the FFmpeg developers
  built with Apple LLVM version 10.0.1 (clang-1001.0.46.4)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.1.4_1 --enable-shared --enable-pthreads --enable-version3 --enable-avresample --cc=clang --host-cflags='-I/Library/Java/JavaVirtualMachines/adoptopenjdk-12.0.1.jdk/Contents/Home/include -I/Library/Java/JavaVirtualMachines/adoptopenjdk-12.0.1.jdk/Contents/Home/include/darwin' --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libmp3lame --enable-libopus --enable-librubberband --enable-libsnappy --enable-libtesseract --enable-libtheora --ena

22805.625 23024.12425


ffmpeg version 4.1.4 Copyright (c) 2000-2019 the FFmpeg developers
  built with Apple LLVM version 10.0.1 (clang-1001.0.46.4)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.1.4_1 --enable-shared --enable-pthreads --enable-version3 --enable-avresample --cc=clang --host-cflags='-I/Library/Java/JavaVirtualMachines/adoptopenjdk-12.0.1.jdk/Contents/Home/include -I/Library/Java/JavaVirtualMachines/adoptopenjdk-12.0.1.jdk/Contents/Home/include/darwin' --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libmp3lame --enable-libopus --enable-librubberband --enable-libsnappy --enable-libtesseract --enable-libtheora --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librtmp --enable-libspeex --enable-videotoolbox --disable-libjack --d

40.91997999999995


frame=  219 fps= 11 q=-1.0 Lsize=    1546kB time=00:00:08.74 bitrate=1447.9kbits/s speed=0.429x    
video:1400kB audio:138kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.512210%
[aac @ 0x7fb92300ae00] Qavg: 235.083
[libx264 @ 0x7fb92300e200] frame I:1     Avg QP:16.11  size: 29491
[libx264 @ 0x7fb92300e200] frame P:55    Avg QP:17.41  size: 13631
[libx264 @ 0x7fb92300e200] frame B:163   Avg QP:21.87  size:  4013
[libx264 @ 0x7fb92300e200] consecutive B-frames:  0.5%  0.0%  2.7% 96.8%
[libx264 @ 0x7fb92300e200] mb I  I16..4: 25.4% 70.4%  4.2%
[libx264 @ 0x7fb92300e200] mb P  I16..4:  4.6% 11.7%  0.3%  P16..4: 23.4%  5.3%  1.5%  0.0%  0.0%    skip:53.2%
[libx264 @ 0x7fb92300e200] mb B  I16..4:  0.3%  0.6%  0.0%  B16..8: 22.1%  1.2%  0.1%  direct: 1.8%  skip:73.9%  L0:49.1% L1:48.5% BI: 2.4%
[libx264 @ 0x7fb92300e200] 8x8 transform intra:70.0% inter:84.3%
[libx264 @ 0x7fb92300e200] coded y,uvDC,uvAC intra: 13.9% 13.5% 2.2% inter: 3.2% 4.4% 0.0%
[libx264 @ 0x7fb9230

In [85]:
final_out = 'videos/output_final.mp4'
if os.path.exists(final_out):
    os.remove(final_out)

str_command = 'ffmpeg '
str_command_args = ' -filter_complex "'
for i in range(len(chosen_items)):
    str_command += ' -i videos/output_'+str(i)+'.mp4'
    str_command_args += '['+str(i)+':v]['+str(i)+':a]'

str_command_args += 'concat=n='+str(len(chosen_items))+':v=1:a=1" -vsync vfr '+final_out
str_command += str_command_args

print(str_command)

ffmpeg  -i videos/output_0.mp4 -i videos/output_1.mp4 -i videos/output_2.mp4 -i videos/output_3.mp4 -filter_complex "[0:v][0:a][1:v][1:a][2:v][2:a][3:v][3:a]concat=n=4:v=1:a=1" -vsync vfr videos/output_final.mp4


In [ ]:
ffmpeg -i video1.mp4 -i video2.mp4 -filter_complex "[0:v][0:a][1:v][1:a]concat=n=2:v=1:a=1" -vsync vfr output.mp4


In [232]:
weights

[77, 56, 135, 27, 187, 105, 195, 186, 161, 130, 255, 152, 63, 116]

In [183]:
def printknapSack(W, wt, val, n):
    K = [[0 for w in range(W + 1)]
            for i in range(n + 1)]
             
    # Build table K[][] in bottom
    # up manner
    for i in range(n + 1):
        for w in range(W + 1):
            if i == 0 or w == 0:
                K[i][w] = 0
            elif wt[i - 1] <= w:
                K[i][w] = max(val[i - 1] 
                  + K[i - 1][w - wt[i - 1]],
                               K[i - 1][w])
            else:
                K[i][w] = K[i - 1][w]
 
    # stores the result of Knapsack
    res = K[n][W]
    print(res)
     
    w = W
    for i in range(n, 0, -1):
        if res <= 0:
            break
        # either the result comes from the
        # top (K[i-1][w]) or from (val[i-1]
        # + K[i-1] [w-wt[i-1]]) as in Knapsack
        # table. If it comes from the latter
        # one/ it means the item is included.
        if res == K[i - 1][w]:
            continue
        else:
 
            # This item is included.
            print(wt[i - 1], i-1)
             
            # Since this weight is included
            # its value is deducted
            res = res - val[i - 1]
            w = w - wt[i - 1]

In [184]:
printknapSack(summary_length, weights, values, len(weights))

tensor(3.7838)
130 9
105 5
27 3
135 2
56 1


In [147]:
dp = []

for i in range(len(chatgpt_summary_sentences)+1):
    row = []
    for j in range(summary_length+1):
        row.append(0)
    dp.append(row)

In [170]:
for i in range(1, len(chatgpt_summary_sentences)+1):
    for j in range(1, summary_length+1):
        maxValWithoutCurr = dp[i - 1][j]
        if j >= weights[i-1]:
            remaining_capacity = j-weights[i-1]
            dp[i][j] = values[i-1] + dp[i-1][remaining_capacity]
            
        dp[i][j] = max(dp[i][j], maxValWithoutCurr)     

In [171]:
dp[-1]

[0,
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.4508),
 tensor(7.

In [37]:
# This is the memoization approach of
# 0 / 1 Knapsack in Python in simple
# we can say recursion + memoization = DP
def knapsack(wt, val, W, n):

    # base conditions
    if n == 0 or W == 0:
        return 0
    if t[n][W] != -1:
        return t[n][W]

    # choice diagram code
    if wt[n-1] <= W:
        t[n][W] = max(
            val[n-1] + knapsack(
                wt, val, W-wt[n-1], n-1),
            knapsack(wt, val, W, n-1))
        return t[n][W]
    elif wt[n-1] > W:
        t[n][W] = knapsack(wt, val, W, n-1)
        return t[n][W]



In [41]:
# Driver code

profit = [60, 100, 120]
weight = [10, 20, 30]
W = 50
n = len(profit)

# We initialize the matrix with -1 at first.
t = [[-1 for i in range(W + 1)] for j in range(n + 1)]
print(np.array(t).shape)
print(knapsack(weight, profit, W, n))
t
# This code is contributed by Prosun Kumar Sarkar

(4, 51)
220


[[-1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1],
 [-1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  60,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  60,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  60],
 [-1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  100,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  160],
 [-1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,

In [188]:
filtered_srt_sentences = []
for l in srt_sentences:
    l_ = l.replace('\n','')
    if l_ == '':
        continue 
    
    if '-->' not in l_:
        if checknum(l_):
            continue
        filtered_srt_sentences.append(l_)
        
final_script = ' '.join(filtered_srt_sentences)

In [189]:
final_script

"Ladies and gentlemen. Please put your hand to gather to welcome all guest of owner Minister for education Mr. Chan Chun Seang A very good morning to gasthof owner Minister for education, Mr. Chan Chee Seng, Keynote Speaker Prof James Lander and Yours President Prof Tan Eng Chai, Deputy President of Academic Affairs and Provost, Prof Aren't and Distinguished Guest, Alumni and Partners Here are Nurses and tuning in online Welcome to future works to point to the second edition of the nus lifelong learning festival. My name is jawi and a proud ulama staff member of nus and ummc for this morning. The theme for this years conference is a infection and we are grateful that so many industry leaders and experts have the general sale of their time today to share their invaluable experiences in this area. We are so grateful that you have taken the time to attend the conference and we hope that you will live with the insights connections and inspiration for our future wife. I. Before we begin a q

In [190]:
final_sentences = []

for sentence in final_script.split("."):
    final_sentences.append(sentence.strip()+".")

In [191]:
final_sentences

['Ladies and gentlemen.',
 'Please put your hand to gather to welcome all guest of owner Minister for education Mr.',
 'Chan Chun Seang A very good morning to gasthof owner Minister for education, Mr.',
 "Chan Chee Seng, Keynote Speaker Prof James Lander and Yours President Prof Tan Eng Chai, Deputy President of Academic Affairs and Provost, Prof Aren't and Distinguished Guest, Alumni and Partners Here are Nurses and tuning in online Welcome to future works to point to the second edition of the nus lifelong learning festival.",
 'My name is jawi and a proud ulama staff member of nus and ummc for this morning.',
 'The theme for this years conference is a infection and we are grateful that so many industry leaders and experts have the general sale of their time today to share their invaluable experiences in this area.',
 'We are so grateful that you have taken the time to attend the conference and we hope that you will live with the insights connections and inspiration for our future wif

# Summarize this transcript "&lt;srt file contents &gt;".  In the summary, use very short sentences and do not use abbreviations and titles

In [66]:
# chat_gpt_summary =  "The transcript is from the opening of an event, likely a conference, where the host introduces key guests, "+\
# "including the Minister for Education, Chan Chun Seng, and other distinguished individuals like keynote speaker James Lander "+\
# "and National University of Singapore President Tan Eng Chai. "+\
# "The event is the second edition of the National University of Singapore Lifelong Learning Festival, with a focus on 'Artificial Intelligence in Action.' "+\
# "The National University of Singapore President emphasizes the transformative power of artificial intelligence, comparing it to the advent of personal  "+\
# "computers and the internet. He discusses the university's efforts in artificial intelligence research, including the establishment of an  "+\
# "artificial intelligence institute aimed at accelerating research and promoting real-world applications. The President also highlights initiatives  "+\
# "to integrate artificial intelligence into teaching, learning, and university operations, including staff training in artificial intelligence and  "+\
# "data literacy. The event is positioned as a critical moment in the artificial intelligence revolution, urging participants to embrace and innovate  "+\
# "with new technologies."

chat_gpt_summary = "The speaker welcomes the audience. "+\
"The event is the NUS Lifelong Learning Festival. "+\
"The theme is AI in action. The focus is on how AI is transforming industries and work. "+\
"NUS has started an AI institute to advance research and partnerships. The institute also addresses "+\
"ethical concerns. NUS is using AI in teaching, learning, and administration. "+\
"Workshops guide staff and students on AI use. Around 5,000 staff completed courses in data literacy and AI"

In [170]:
chatgpt_summary_sentences = []

for sentence in chat_gpt_summary.split("."):
    chatgpt_summary_sentences.append(sentence.strip()+".")

In [171]:
chatgpt_summary_sentences

['The speaker welcomes the audience.',
 'The event is the NUS Lifelong Learning Festival.',
 'The theme is AI in action.',
 'The focus is on how AI is transforming industries and work.',
 'NUS has started an AI institute to advance research and partnerships.',
 'The institute also addresses ethical concerns.',
 'NUS is using AI in teaching, learning, and administration.',
 'Workshops guide staff and students on AI use.',
 'Around 5,000 staff completed courses in data literacy and AI.']

In [200]:
# Find time lengths

with open("NUS-FutureWork-first10.json") as f:
    speechmatic_json = json.load(f)
f.close()

In [201]:

# # One way - Needs more work

# time_segs = []
# for word_ind, word in enumerate(speechmatic_json['results']):
#     if 'is_eos' in word and word['is_eos']:
#         time_segs.append(word['end_time'] - word['start_time'])
# len(time_segs), len(final_sentences)

In [216]:
start_times = []
end_times = []
time_segs = []
final_timed_sentences = []
cnt = 0
for ind, final_sentence in enumerate(final_sentences):
    final_sentence_tokens = []
    start_time = 0
    end_time = 0
    for final_sentence_token in final_sentence.split(' '):
        if final_sentence_token.endswith(','):
            final_sentence_tokens.append(final_sentence_token.replace(',',''))
            final_sentence_tokens.append(',') #separate comma token
        else:
            final_sentence_tokens.append(final_sentence_token)
        
    # for tk in final_sentence_tokens:
    if len(final_sentence_tokens) > 5:
        print(final_sentence_tokens[0], final_sentence_tokens[1], final_sentence_tokens[2], final_sentence_tokens[3], final_sentence_tokens[4],'|', final_sentence_tokens[-1].split('.')[0])
        for word_ind, word in enumerate(speechmatic_json['results']):
            if word['alternatives'][0]['content'].lower() == final_sentence_tokens[0].lower() and \
                speechmatic_json['results'][word_ind+1]['alternatives'][0]['content'].lower() == final_sentence_tokens[1].lower() and \
                speechmatic_json['results'][word_ind+2]['alternatives'][0]['content'].lower() == final_sentence_tokens[2].lower() and \
                speechmatic_json['results'][word_ind+3]['alternatives'][0]['content'].lower() == final_sentence_tokens[3].lower() and \
                speechmatic_json['results'][word_ind+4]['alternatives'][0]['content'].lower() == final_sentence_tokens[4].lower():
                print(word['start_time'], word['end_time'])
                time_segs.append(word['end_time'] - word['start_time'])
                final_timed_sentences.append(final_sentence)
                start_times.append(word['start_time'])
                end_times.append(word['end_time'])
        print(ind, '----')
len(time_segs), len(final_timed_sentences)        

Please put your hand to | Mr
9.83 10.31
1 ----
Chan Chun Seang A very | Mr
15.89 16.28
2 ----
Chan Chee Seng , Keynote | festival
68.47 68.71
3 ----
My name is jawi and | morning
92.59 92.8
4 ----
The theme for this years | area
99.37 99.52
5 ----
We are so grateful that | wife
113.59 113.77
6 ----
Before we begin a quick | briefing
124.72 125.05
8 ----
In case of emergency Please | state
128.71 128.86
9 ----
Or at the back of | instructions
134.32 134.47
10 ----
We would like to invite | Thanks
144.04 144.13
12 ----
Chan Chuncheon , the Guest | Friends
168.4 168.64
14 ----
Good morning and I bet | table
173.44 173.62
15 ----
The theme for this year's | action
184.14 184.29
16 ----
Many of us have already | action
195.58 195.88
18 ----
We will have the privilege | deploy
213.97 214.18
20 ----
The changes are as broad | journey
227.29 227.47
21 ----
We are in the watershed | revolution
237.76 237.91
22 ----
Some of US here to | period
244.57 244.93
23 ----
Nearly forty years ago where |

(34, 34)

In [217]:
for sent_ind, sent in enumerate(final_timed_sentences):
    print(time_segs[sent_ind], sent)

0.4800000000000004 Please put your hand to gather to welcome all guest of owner Minister for education Mr.
0.39000000000000057 Chan Chun Seang A very good morning to gasthof owner Minister for education, Mr.
0.23999999999999488 Chan Chee Seng, Keynote Speaker Prof James Lander and Yours President Prof Tan Eng Chai, Deputy President of Academic Affairs and Provost, Prof Aren't and Distinguished Guest, Alumni and Partners Here are Nurses and tuning in online Welcome to future works to point to the second edition of the nus lifelong learning festival.
0.20999999999999375 My name is jawi and a proud ulama staff member of nus and ummc for this morning.
0.14999999999999147 The theme for this years conference is a infection and we are grateful that so many industry leaders and experts have the general sale of their time today to share their invaluable experiences in this area.
0.1799999999999926 We are so grateful that you have taken the time to attend the conference and we hope that you will

In [59]:
# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

In [210]:
chatgpt_embeddings = model.encode(chatgpt_summary_sentences)
final_sentences_embeddings = model.encode(final_sentences)

In [211]:
similarities = model.similarity(chatgpt_embeddings, final_sentences_embeddings) # Not used


cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
cos_similarities = []
for chatgpt_embedding in chatgpt_embeddings:
    chatgpt_sim = []
    for final_sentences_embedding in final_sentences_embeddings:
        chatgpt_sim.append(cos(torch.from_numpy(chatgpt_embedding), torch.from_numpy(final_sentences_embedding)).numpy())
    cos_similarities.append(chatgpt_sim)

In [212]:
for i in range(len(chatgpt_summary_sentences)):
    print(chatgpt_summary_sentences[i], '==>', final_sentences[np.argmax(cos_similarities[i])])
    print('--------')

The speaker welcomes the audience. ==> We would like to invite NS President Prof Tan Eng Chai to give the Welcome Actress Thanks.
--------
The event is the NUS Lifelong Learning Festival. ==> The theme for this year's learning Festival is future work to point zero i in action.
--------
The theme is AI in action. ==> We are actively positioning ourselves to right the potential of ai On the Research Front.
--------
The focus is on how AI is transforming industries and work. ==> The changes are as broad about and how They are bringing the workforce on this new ai journey.
--------
NUS has started an AI institute to advance research and partnerships. ==> We are actively positioning ourselves to right the potential of ai On the Research Front.
--------
The institute also addresses ethical concerns. ==> This institute will also conduct research on how to regulate And to safeguard Transparency and Accountability.
--------
NUS is using AI in teaching, learning, and administration. ==> For exam

In [195]:
final_sentences

['Ladies and gentlemen.',
 'Please put your hand to gather to welcome all guest of owner Minister for education Mr.',
 'Chan Chun Seang A very good morning to gasthof owner Minister for education, Mr.',
 "Chan Chee Seng, Keynote Speaker Prof James Lander and Yours President Prof Tan Eng Chai, Deputy President of Academic Affairs and Provost, Prof Aren't and Distinguished Guest, Alumni and Partners Here are Nurses and tuning in online Welcome to future works to point to the second edition of the nus lifelong learning festival.",
 'My name is jawi and a proud ulama staff member of nus and ummc for this morning.',
 'The theme for this years conference is a infection and we are grateful that so many industry leaders and experts have the general sale of their time today to share their invaluable experiences in this area.',
 'We are so grateful that you have taken the time to attend the conference and we hope that you will live with the insights connections and inspiration for our future wif

In [83]:
len(speechmatic_json['results'])

959